# Analyser data fra kompensasjonsordningen med Python og Pandas

I denne notatboken brukes data om tildelinger av støtte fra Kompensasjonsordningen for næringslivet. For oversikt over hva som finnes av Åpne data fra Brønnøysundregistrene, gå til [https://data.brreg.no]. På [https://data.norge.no] finner dere oversikt over data fra både Brønnøysundregistrene og andre.

Dataene vi skal bruke CSV-filer i dette repositoriet, men inneholder ikke de siste tallene. Vedtak om støtte etter kompensasjonsordnigen skjer løpende. Oppdaterte data legges ut hver time. Dette repo-et inneholder en kopi av dataene fra mars i år. Ønsker du en oppdatert versjon kan du laste ned med f.eks. curl:

```
curl https://data.brreg.no/kompensasjonsordning/api/tilskudd/csv --output tilskudd.csv
```

Det er også mulig å laste ned JSON hvis noen foretrekker det, det gjelder også de andre filene vi laster ned senere:

```
curl https://data.brreg.no/kompensasjonsordning/api/tilskudd/json --output tilskudd.json

# Installer Pandas

Dersom du ikke har gjort det allerede, installer Pandas (men husk den obligatoriske oppgraderingen av pip først). 

In [ ]:
%pip install --upgrade pip
%pip install pandas

## Last inn datasettet og gjør deg kjent

Vi starter med å importere pandas, lese inn csv-filen, og få et inntrykk av hva den inneholder. Merk at CSV-fila bruker semikolon som skilletegn.




In [ ]:
import pandas as pd



In [ ]:
df = pd.read_csv('tilskudd.csv', sep=';')
df.head()


Vi kan også få et uttrekk av helt tilfeldige rader, istedenfor bare de første, ved hjelp av sample(n) der n er antall rader:

In [ ]:
df.sample(10)

Med head() og sample(n) får vi kjapt et inntrykk av dataene vi har å gjøre med. Det er en liste over tilskudd, hvem som har mottatt, hvor de holder til med mer.

Siden tabellen er så bred, kan det også være nyttig å bruke columns for å få en liste over alle kolonnenavnene:


In [ ]:
df.columns

## Litt lynkjapp statistikk med describe() avslører noen svakheter
Pandas har funksjon for å gi en rask oppsummering av noen statistiske egenskaper ved dataene i en dataframe, gjennom describe(). Da får vi blant annet antall verdier, median, minimum, maximum og gjennomsnitt (50 %-verdien):

In [ ]:
df.describe()

Men her ser vi at det som det er mest interessant å få frem verdier på er ikke med, for eksempel beløpene for tilskudd og omsetningstall. Det skyldes at disse verdiene leses inn som tekst istedenfor tall, pga komma som desimaltegn. Til gjengjeld får vi masse statistiske verdier for numre som er identifikatorer og ikke tall, f.eks. organisasjonsnummer, næringskode, kommunenummer. For å bekrefte det kan vi se nærmere på en enkelt kolonne, og se hvilken type (dtype) den har:


In [ ]:
df['tilskuddsbelop']


In [ ]:
df['tilskuddsnummer']

Beløp er altså den generelle "object"-typen mens identifikatoren tilskuddsnummer er tall (int64)

Årsaken til at beløp blir tekst er at desimalskilletegnet i Pandas er punktum, mens vi bruker komma i Norge. Så for ved å presisere dette når vi leser inn Excel-fila igjen kan vi få rettet opp noen av datatypene:

In [ ]:
df = pd.read_csv('tilskudd.csv', sep=';', decimal=',')

In [ ]:
df.describe()

Nå ser vi at beløpene er blitt tall, og til gjengjeld er næringskodene ikke lenger tall. Det hjalp litt. 

In [ ]:
df['tilskuddsbelop']

In [ ]:
df['tilskuddsbelop'].describe().round(2)

I tillegg kan vi endre datatypen til de ulike numrene som egentlig er identifikatorer.

In [ ]:
# TODO

# Identfisere enkeltselskap og hente mer detaljer
Hvis jeg f.eks. gjerne vil finne ut hvem som har fått minst, kan jeg spørre etter den som har tilskuddsbeløp lik minimumsbeløpet:

In [ ]:
df[df['tilskuddsbelop']==5000.34] # minimumsbeløpet kan være et annet med oppdaterte tall, sjekk resultatet i cellen over

Når vi har organisasjonsnummeret til en konkret virksomhet, kan vi få mer informasjon om denne virksomheten  gjennom API-et for åpne data fra Enhetsregisteret. Da bruker request-biblioteket.

In [ ]:
%pip install requests

In [ ]:
import requests

Først lagrer vi URL-en til API-et, og gjør et oppslag til URL-en med tillegget av organisasjonsnummeret, og skriver ut resultatet.

In [ ]:
ERAPIURL = 'https://data.brreg.no/enhetsregisteret/api/enheter/'
r = requests.get(ERAPIURL + '982575044')
r.text

Det er også et eget API for å hente nøkkeltall fra det siste innsendte årsregnskapet:

In [ ]:
REGNSKAPAPIURL = 'http://34.98.91.231/regnskap/'
r_regnskap = requests.get(REGNSKAPAPIURL + '982575044')
r_regnskap.text

Vi kan også åpne nettsiden med informasjon om enheten (NB! Dette fungerer når jeg kjører lokalt, men jeg fikk det ikke til å fungere ved bruk av binder, men det er ikke kritisk. Hvis dere er er nysgjerrige kan dere sette sammen linken manuelt og sjekke resultatet selv ved å lime det inn i en ny nettleserfane :-))

In [ ]:
import webbrowser

ERWEBURL = 'https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr='
# generate an URL
webbrowser.open(ERWEBURL + '982575044')

## Næringskoder
Det er tre kolonner med det vi kaller "næringskoder". Det er en kode Brønnøysundregistrene fastsetter basert på hva som er bedriftenes formål. Næringskoden har til hensikt å gjøre det mulig for SSB å lage statistikk. I andre land er det bedriftene selv som velger næringskode. Brønnøysundregistrene har tatt i bruk maskinlæring for å hjelpe saksbehandlerne med å fastsette næringskode, og de som er interesserte kan se mer om dette her:
1) demo-grensesnitt for å få forslag til næringskode
2) github med kode

Det er SSB som forvalter næringskodene, og på deres side [Standard for næringsgruppering (SN)](https://www.ssb.no/klass/klassifikasjoner/6) kan dere lese mer om kodene. Her kan dere også laste ned en fil med alle næringskode og beskrivelse av dem. Den skal vi bruke videre. Den ligger i repoet med tittelen nace.csv. Hvis dere vil laste den ned selv, er URL-en https://data.ssb.no/api/klass/v1//versions/30.csv?language=nb 

Merk valg for separator og encoding når vi leser inn fila. Og som dere ser av utvalget dere får opp med "sample" inneholder den nye dataframen norske navn på kodene i kolonnen 'name'.

In [ ]:
df_nace = pd.read_csv('nace.csv', sep=';', encoding='iso8859-1')
df_nace.sample(5)

In [ ]:
df_nace['code']

Det er den første kolonnen, code, som inneholder alle kodene. I vår opprinnelige dataframe med alle tilskuddene er det tre ulike næringskoder, men den første representerer hovedaktiviteten, så vi bryr oss bare om den. For å gjøre det enklere for oss, lager vi en ny dataframe, som sier litt om innholdet, og endrer samtidig navn på den kolonnene med næringskoder vi er interessert i til samme som i df_nace. På den måten kan vi enklere koble informasjonen mellom de to:

In [ ]:
df_tilskudd = df.rename(columns={'naeringskode1':'code'})

Nå som vi har to dataframes, som begge har en kolonne "code", kan vi bruke "merge"-funksjonen til å slå innholdet sammen. 

Situasjonen før er at vi bare har en kode av typen '93.130' som vi ikke vet hva betyr:

In [ ]:
df_tilskudd.head()

Så kombinerer vi de to dataframene, og dermed får vi tekster for næringskodene i kolonnen 'name':

In [ ]:
df_extended = pd.merge(df_tilskudd, df_nace)
df_extended.sample(5)

Da kan vi se nærmere på tilskudd til enkelte næringskoder, f.eks. '93.130'

In [ ]:
df_extended[df_extended['code']=='93.130'].sample(5)

In [ ]:
df_extended[df_extended['code']=='93.130'].describe().round(2)

## Kombinere med andre data fra Enhetsregisteret: Antall ansatte

Det er også mulig å laste ned hele enhetsregisteret. Da får dere tilgang til mer detaljer om de enkelte selskapene, for eksempel antall ansatte. En oppgave kan være å finne ut hvor mange av bedriftene som har fått støtte gjennom kompensasjonsordningen har færre enn 10 ansatte?

Ettersom Enhetsregisteret er ganske stort så har vi "juksa litt" og laget en forenklet versjon som kun har organisasjonsnummer og antall ansatte. Men nederst i denne notatboken står oppskriften for å gjøre dette på egenhånd. Det kan hende at det er begrensninger i Jupyter for størrelse på fila som leses inn, så det må kanskje gjøres i "vanlig" Python.

Fila heter orgnr_ansatte.csv

In [ ]:

df_enheter = pd.read_csv('orgnr_ansatte.csv')

In [ ]:
df_enheter.columns

In [ ]:
df_enheter.sample(10)

Nå har vi en dataframe med alle tilskuddene, men den mangler data om antall ansatte. Og vi har en dataframe med antall ansatte pr organisasjonsnummer. Så da trenger vi å "merge" sånn at vi får lagt til antall ansatte i dataframen med tilskudd.

Denne gangen, istedenfor å endre så det er identisk navn på kolonnene vi skal "merge" på, så kan vi også angi eksplisitt hvilken kolonne i hver av dataframene som er nøkkelen:

In [ ]:
df_extended2 = pd.merge(df_extended, df_enheter, left_on='tilskuddsmottakerOrganisasjonsnummer', right_on='organisasjonsnummer')

Vi tar en titt på resultatet:

In [ ]:
df_extended2

In [ ]:
df_extended2[['tilskuddsmottakerNavn','antallAnsatte']]

In [ ]:
df_extended2['antallAnsatte'].describe().round(0)

In [ ]:
df_extended2[['tilskuddsmottakerNavn','antallAnsatte']][df_extended2['antallAnsatte']<10]

## Laste ned og lage forenklet versjon av Enhetsregisteret
Som nevnt over


URL til nedlasting av alle hovedenheter: https://data.brreg.no/enhetsregisteret/api/enheter/lastned
Resultatet er en zippet fil, som f.eks. kan unzippes med gunzip <filnavn>

Koden nedenfor tar utgangspunkt i at filnavnet er enheter_alle.json
```
import pandas as pd
df_enheter = pd.read_json('enheter_alle.json')  # df med hele Enhetsregisteret
df_orgnrOgAnsatte = df_enheter[['organisasjonsnummer','antallAnsatte']]  # lager ny df kun med to kolonner
df_orgnrOgAnsatte.to_csv('orgnr_ansatte.csv')  # lager en CSV-fil med innholdet i den nye df-en

